In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

In [2]:
# Suppress warnings about processing individual user vectors as opposed to a collection of users in a dataframe
import warnings
warnings.filterwarnings(action='ignore')

# Import Data #

In [3]:
# Import data from file
df_o = pd.read_csv("../Datasets/moocs23-24.csv")
df_o

,Id,institute,course_id,year,semester,userid_DI,viewed,explored,certified,final_cc_cname_DI,...,grade,start_time_DI,last_event_DI,nevents,ndays_act,nplay_video,nchapters,nforum_posts,incomplete_flag,age
0,4,HarvardX,PH207x,2023,Fall,MHxPC130313697,0,0,0,India,...,0.0,7/24/2023,7/27/2024,6,3,197757,0,0,0,23
1,6,HarvardX,PH207x,2023,Fall,MHxPC130237753,1,0,0,United States,...,0.0,7/24/2023,12/24/2023,107,8,7,2,0,0,19
2,7,HarvardX,CS50x,2023,Summer,MHxPC130202970,1,0,0,United States,...,0.0,7/24/2023,3/28/2024,8,1,197757,1,0,0,24
3,20,HarvardX,CS50x,2023,Summer,MHxPC130223941,1,0,0,Other Middle East/Central Asia,...,0.0,7/24/2023,7/15/2024,25,2,197757,4,0,0,20
4,22,HarvardX,PH207x,2023,Fall,MHxPC130317399,0,0,0,Australia,...,0.0,7/24/2023,8/25/2023,3,2,197757,0,0,0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416916,583001,MITx,6.002x,2024,Spring,MHxPC130030805,1,0,0,Pakistan,...,0.0,9/7/2024,9/7/2024,29,1,197757,1,0,0,24
416917,583002,MITx,6.00x,2024,Spring,MHxPC130184108,1,0,0,Canada,...,0.0,9/7/2024,9/7/2024,97,1,4,2,0,0,22
416918,583003,MITx,6.00x,2024,Spring,MHxPC130359782,0,0,0,Other Europe,...,0.0,9/7/2024,9/7/2024,1,1,197757,0,0,0,22
416919,583004,MITx,6.002x,2024,Spring,MHxPC130098513,0,0,0,United States,...,0.0,9/7/2024,9/7/2024,1,1,197757,0,0,0,34


In [4]:
# df_o.columns

In [5]:
# Create copy of the dataframe to preserve integrity
df = df_o.copy()

# Data Cleaning #

### *Check columns for N/A values* ###

In [6]:
df.isna().sum()

Id                       0
institute                0
course_id                0
year                     0
semester                 0
userid_DI                0
viewed                   0
explored                 0
certified                0
final_cc_cname_DI        0
LoE_DI                   0
gender               23211
grade                    0
start_time_DI            0
last_event_DI            0
nevents                  0
ndays_act                0
nplay_video              0
nchapters                0
nforum_posts             0
incomplete_flag          0
age                      0
dtype: int64

In [7]:
# print("Percentage of entries that lack gender information: ",format(23211/len(df_o)*100, ".2f"), "%")

Percentage of entries that lack gender information:  5.57 %

Because gender is one key feature of this dataset and because it is the only aspect of the dataset that appears to be missing, it makes sense to remove the rows with missing values in order to reduce the amount of "noise" in the data itself.

In [8]:
# Isolate indices of rows with blank values and drop rows
df = df.drop(df[df["gender"].isna()].index)

In [9]:
# Reset index after dropping rows
df = df.reset_index(drop=True)

In [10]:
df

,Id,institute,course_id,year,semester,userid_DI,viewed,explored,certified,final_cc_cname_DI,...,grade,start_time_DI,last_event_DI,nevents,ndays_act,nplay_video,nchapters,nforum_posts,incomplete_flag,age
0,4,HarvardX,PH207x,2023,Fall,MHxPC130313697,0,0,0,India,...,0.0,7/24/2023,7/27/2024,6,3,197757,0,0,0,23
1,6,HarvardX,PH207x,2023,Fall,MHxPC130237753,1,0,0,United States,...,0.0,7/24/2023,12/24/2023,107,8,7,2,0,0,19
2,7,HarvardX,CS50x,2023,Summer,MHxPC130202970,1,0,0,United States,...,0.0,7/24/2023,3/28/2024,8,1,197757,1,0,0,24
3,20,HarvardX,CS50x,2023,Summer,MHxPC130223941,1,0,0,Other Middle East/Central Asia,...,0.0,7/24/2023,7/15/2024,25,2,197757,4,0,0,20
4,22,HarvardX,PH207x,2023,Fall,MHxPC130317399,0,0,0,Australia,...,0.0,7/24/2023,8/25/2023,3,2,197757,0,0,0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393705,583001,MITx,6.002x,2024,Spring,MHxPC130030805,1,0,0,Pakistan,...,0.0,9/7/2024,9/7/2024,29,1,197757,1,0,0,24
393706,583002,MITx,6.00x,2024,Spring,MHxPC130184108,1,0,0,Canada,...,0.0,9/7/2024,9/7/2024,97,1,4,2,0,0,22
393707,583003,MITx,6.00x,2024,Spring,MHxPC130359782,0,0,0,Other Europe,...,0.0,9/7/2024,9/7/2024,1,1,197757,0,0,0,22
393708,583004,MITx,6.002x,2024,Spring,MHxPC130098513,0,0,0,United States,...,0.0,9/7/2024,9/7/2024,1,1,197757,0,0,0,34


In [11]:
# Remove columns that are not needed for analysis
df = df.drop(["Id","year", "semester", "start_time_DI", "last_event_DI","viewed","grade","start_time_DI","explored","last_event_DI","nevents","ndays_act","nplay_video", "nchapters", "nforum_posts","incomplete_flag"], axis=1)

### *Correct User Anomalies* ###
In some cases users across institutes have the same number but different user features, which leads me to believe that I ought to separate the user ID among institutes.

In [12]:
# Set character to denote institute ('M' or 'H')
df["institute_char"] = df["institute"].apply(lambda x: "H" if x=="HarvardX" else "M")

In [13]:
# Set character to denote institute ('m' or 'h')
# df["institute_char"] = ['H' if institute=="HarvardX" else 'M' for institute in  df["institute"]]

In [14]:
# Zip while removing preceeding character for legibility
userids = [str(item[1]+item[0][5:]) for item in list(zip(df["userid_DI"], df["institute_char"]))] 
df["userid_DI"]= userids

In [15]:
df.drop(["institute_char"], axis=1, inplace=True)

In [16]:
df

,institute,course_id,userid_DI,certified,final_cc_cname_DI,LoE_DI,gender,age
0,HarvardX,PH207x,H130313697,0,India,Bachelor's,m,23
1,HarvardX,PH207x,H130237753,0,United States,Secondary,m,19
2,HarvardX,CS50x,H130202970,0,United States,Bachelor's,m,24
3,HarvardX,CS50x,H130223941,0,Other Middle East/Central Asia,Secondary,m,20
4,HarvardX,PH207x,H130317399,0,Australia,Master's,f,32
...,...,...,...,...,...,...,...,...
393705,MITx,6.002x,M130030805,0,Pakistan,Master's,m,24
393706,MITx,6.00x,M130184108,0,Canada,Bachelor's,m,22
393707,MITx,6.00x,M130359782,0,Other Europe,Bachelor's,f,22
393708,MITx,6.002x,M130098513,0,United States,Doctorate,m,34


# Feature Engineering #

### *Transform categorical information to one-hot encoding* ###
#### *Gender* ####

In [17]:
# Transform gender information to binary representation
# https://www.geeksforgeeks.org/python/using-apply-in-pandas-lambda-functions-with-multiple-if-statements/
df["gender_numerical"] = df["gender"].apply(lambda x: 1 if x=='m' else 0)

In [18]:
# Drop original gender column
df = df.drop(["gender"], axis=1)

# Check results
# df.head()

#### *Age* ####

In [19]:
# Create a set of 4 age brackets (indicators) and assign category to age bracket
# https://www.geeksforgeeks.org/python/using-apply-in-pandas-lambda-functions-with-multiple-if-statements/
df["age_indicator"] = df["age"].apply(lambda x: 0 if x<=17 else (
                                                        1 if x>17 and x<=35 else (
                                                        2 if x>35 and x<60 else 4)))

In [20]:
# Produce age dummy enoding based on indicator information
age_dummies = pd.get_dummies(df["age_indicator"], prefix="age_indicator")

# Check output
# age_dummies

In [21]:
# Concatenate dummies to original df
df = pd.concat([df, age_dummies], axis=1)

# Check result
# df.head()

#### *Level of Education* ####

In [22]:
# Generate dummies
LoE_dummies = pd.get_dummies(df["LoE_DI"], prefix="education")

# Check output
# LoE_dummies

In [23]:
# # Concatenate educational econding to original df
df = pd.concat([df, LoE_dummies], axis=1)

# Check results
# df.head()

#### *Geographical Information* ####

In [24]:
# Get regional information dummies
reg_dummies = pd.get_dummies(df["final_cc_cname_DI"], prefix="region")

# Check output
# reg_dummies

In [25]:
# # Concatenate regional econding to original df
df = pd.concat([df, reg_dummies], axis=1)

# Check results
# df.head()

#### *Drop Original Columns* ####

In [26]:
# # Make a copy of the processed dataset before dropping columns
df_final = df.copy()

In [27]:
# Drop columns that do not have processed information
df_final = df_final.drop(["final_cc_cname_DI", "LoE_DI","age","age_indicator"], axis=1)

In [28]:
# Check dataframe
df_final

,institute,course_id,userid_DI,certified,gender_numerical,age_indicator_0,age_indicator_1,age_indicator_2,age_indicator_4,education_Bachelor's,...,region_Pakistan,region_Philippines,region_Poland,region_Portugal,region_Russian Federation,region_Spain,region_Ukraine,region_United Kingdom,region_United States,region_Unknown/Other
0,HarvardX,PH207x,H130313697,0,1,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,HarvardX,PH207x,H130237753,0,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,HarvardX,CS50x,H130202970,0,1,False,True,False,False,True,...,False,False,False,False,False,False,False,False,True,False
3,HarvardX,CS50x,H130223941,0,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,HarvardX,PH207x,H130317399,0,0,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393705,MITx,6.002x,M130030805,0,1,False,True,False,False,False,...,True,False,False,False,False,False,False,False,False,False
393706,MITx,6.00x,M130184108,0,1,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
393707,MITx,6.00x,M130359782,0,0,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
393708,MITx,6.002x,M130098513,0,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [29]:
# Verify columns
# df_final.columns

### *Separate users who completed multiple courses for further processing* ###

In [30]:
# Find users who have completed multiple courses
user_completion_data = df_final[df_final["certified"]==1]["userid_DI"].value_counts()

In [31]:
# Define list to hold user IDs for multi-course users
multi_course_users = []

# Filter multi-course users
for idx, val in enumerate(user_completion_data):
    if (val > 1):
         # Add user ID to list
        multi_course_users.append(user_completion_data.index[idx])   

# Check size of multi-course user list
len(multi_course_users)

497

Apportion the first 80% of the users to the train set and the remainder to the test set

In [32]:
# Apportion the first 80% of the users to the train set
train_end_idx = int(len(multi_course_users)*0.8)

# Create a dataframe to be appended to the trainig dataframe
train_multi_course_user_df = df_final[df_final["userid_DI"].isin(multi_course_users[:train_end_idx])]

# Create a dataframe to be appended to the testing dataframe
test_multi_course_user_df = df_final[df_final["userid_DI"].isin(multi_course_users[train_end_idx:])]

# Check proportion
# print(len(train_multi_course_user_df), len(test_multi_course_user_df))

Remove these users from the general dataframe before creating the train-test split

In [33]:
# Create a DF with all multi users
multi_course_user_df = df_final[df_final["userid_DI"].isin(multi_course_users)]

# Drop all multi-course users from dataframe
df_final.drop(multi_course_user_df.index, axis=0, inplace=True)

In [34]:
# Check results
df_final = df_final.reset_index(drop=True)
df_final

,institute,course_id,userid_DI,certified,gender_numerical,age_indicator_0,age_indicator_1,age_indicator_2,age_indicator_4,education_Bachelor's,...,region_Pakistan,region_Philippines,region_Poland,region_Portugal,region_Russian Federation,region_Spain,region_Ukraine,region_United Kingdom,region_United States,region_Unknown/Other
0,HarvardX,PH207x,H130313697,0,1,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,HarvardX,PH207x,H130237753,0,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,HarvardX,CS50x,H130202970,0,1,False,True,False,False,True,...,False,False,False,False,False,False,False,False,True,False
3,HarvardX,CS50x,H130223941,0,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,HarvardX,PH207x,H130317399,0,0,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392526,MITx,6.002x,M130030805,0,1,False,True,False,False,False,...,True,False,False,False,False,False,False,False,False,False
392527,MITx,6.00x,M130184108,0,1,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
392528,MITx,6.00x,M130359782,0,0,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
392529,MITx,6.002x,M130098513,0,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [35]:
df_final.columns

Index(['institute', 'course_id', 'userid_DI', 'certified', 'gender_numerical',
       'age_indicator_0', 'age_indicator_1', 'age_indicator_2',
       'age_indicator_4', 'education_Bachelor's', 'education_Doctorate',
       'education_Less than Secondary', 'education_Master's',
       'education_Secondary', 'region_Australia', 'region_Bangladesh',
       'region_Brazil', 'region_Canada', 'region_China', 'region_Colombia',
       'region_Egypt', 'region_France', 'region_Germany', 'region_Greece',
       'region_India', 'region_Indonesia', 'region_Japan', 'region_Mexico',
       'region_Morocco', 'region_Nigeria', 'region_Other Africa',
       'region_Other East Asia', 'region_Other Europe',
       'region_Other Middle East/Central Asia',
       'region_Other North & Central Amer., Caribbean', 'region_Other Oceania',
       'region_Other South America', 'region_Other South Asia',
       'region_Pakistan', 'region_Philippines', 'region_Poland',
       'region_Portugal', 'region_Russian Fed

In [36]:
# Check again for duplciates
# user_completion_data = df_final[df_final["certified"]==1]["userid_DI"].value_counts()
# user_completion_data

# Create Test-Train Split #

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
# Split into training and testing data
# REVISED!!! Users who are taking multiple courses are not being filtered out because those users provide
#  an opportunity to see whether users are matched/clustered correctly

# Stratify data by both course ID and certification information
train_df_o, test_df_o = train_test_split(df_final, test_size=0.2, random_state=7,stratify=df_final[["course_id", "certified"]])

In [39]:
len(train_df_o)

314024

In [40]:
len(test_df_o)

78507

In [41]:
# Add multi-course train users to the train_df_o database
train_df = pd.concat([train_df_o, train_multi_course_user_df], axis = 0)
train_df = train_df.reset_index(drop=True)
train_df

,institute,course_id,userid_DI,certified,gender_numerical,age_indicator_0,age_indicator_1,age_indicator_2,age_indicator_4,education_Bachelor's,...,region_Pakistan,region_Philippines,region_Poland,region_Portugal,region_Russian Federation,region_Spain,region_Ukraine,region_United Kingdom,region_United States,region_Unknown/Other
0,MITx,6.00x,M130414773,0,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,MITx,3.091x,M130240725,0,1,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,HarvardX,PH207x,H130082344,0,1,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,MITx,6.00x,M130442605,0,1,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
4,HarvardX,CB22x,H130226407,0,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314971,MITx,3.091x,M130241907,0,1,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
314972,MITx,7.00x,M130241907,1,1,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
314973,MITx,6.002x,M130205207,1,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
314974,MITx,7.00x,M130205207,1,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [42]:
# train_df.loc[train_df["userid_DI"] == "MHxPC130307522"].index

In [43]:
# train_df.iloc[314995]

In [44]:
# Add multi-course test users to the test_df_o database
test_df = pd.concat([test_df_o, test_multi_course_user_df], axis = 0)
test_df = test_df.reset_index(drop=True)

# Create Course Completion Matrices #

In [45]:
# Create a course-completion matrix for the training subset of the data
train_pt = pd.pivot_table(train_df, index=["userid_DI"], columns="course_id", values="certified", aggfunc="sum")

In [46]:
# Sum up values for each user to help with further processing
train_pt["sum"] = [np.nansum(train_pt.loc[row,:].values) for row in train_pt.index]

In [47]:
# Check outputs
train_pt

course_id,14.73x,2.01x,3.091x,6.002x,6.00x,7.00x,8.02x,8.MReV,CB22x,CS50x,ER22x,PH207x,PH278x,sum
userid_DI,,,,,,,,,,,,,,
H130000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2.0
H130000006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0
H130000007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
H130000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
H130000026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M130597660,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
M130597662,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
M130597664,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [48]:
# Create a course-completion matrix for the test subset of the data
test_pt = pd.pivot_table(test_df, index=["userid_DI"], columns="course_id", values="certified", aggfunc="sum")

In [49]:
# Sum up values for each user to help with further processing
test_pt["sum"] = [np.nansum(test_pt.loc[row,:].values) for row in test_pt.index]

In [50]:
# Check outputs
test_pt


course_id,14.73x,2.01x,3.091x,6.002x,6.00x,7.00x,8.02x,8.MReV,CB22x,CS50x,ER22x,PH207x,PH278x,sum
userid_DI,,,,,,,,,,,,,,
H130000016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
H130000021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
H130000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
H130000035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
H130000059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M130597652,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
M130597662,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
M130597663,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## K-Means clustering ##
Train the k-means clustering algorithm using k=75 (see results of elbow and silhouette tests in the report). Information can also be found in the EDA file.

In [51]:
# K-means clustering
# https://lisalondon.medium.com/applying-k-means-clustering-model-to-customer-segmentation-4254386c7563
# https://www.youtube.com/watch?v=afPJeQuVeuY
# https://www.youtube.com/watch?v=ZzJ13-6kCAY
# https://medium.com/analytics-vidhya/how-to-determine-the-optimal-k-for-k-means-708505d204eb


In [52]:
# Import necessary libraries

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [53]:
# Check columns in order to define parameters
train_df.columns

Index(['institute', 'course_id', 'userid_DI', 'certified', 'gender_numerical',
       'age_indicator_0', 'age_indicator_1', 'age_indicator_2',
       'age_indicator_4', 'education_Bachelor's', 'education_Doctorate',
       'education_Less than Secondary', 'education_Master's',
       'education_Secondary', 'region_Australia', 'region_Bangladesh',
       'region_Brazil', 'region_Canada', 'region_China', 'region_Colombia',
       'region_Egypt', 'region_France', 'region_Germany', 'region_Greece',
       'region_India', 'region_Indonesia', 'region_Japan', 'region_Mexico',
       'region_Morocco', 'region_Nigeria', 'region_Other Africa',
       'region_Other East Asia', 'region_Other Europe',
       'region_Other Middle East/Central Asia',
       'region_Other North & Central Amer., Caribbean', 'region_Other Oceania',
       'region_Other South America', 'region_Other South Asia',
       'region_Pakistan', 'region_Philippines', 'region_Poland',
       'region_Portugal', 'region_Russian Fed

In [54]:
# Define parameters for K-Means algorithm. Set random state in order to create repeatable results.
kmeans = KMeans(n_clusters=75, max_iter = 1000, random_state=7)

# Create clusters and store labels
cluster_labels = kmeans.fit_predict(train_df.iloc[:,4:]) # select columns 4 and onwards

In [55]:
# Show cluster labels
cluster_labels

array([57,  7, 31, ..., 13, 13, 13], dtype=int32)

In [56]:
# Check distribution
# sns.histplot(cluster_labels)


In [57]:
# Show cluster centre information
# kmeans.cluster_centers_

In [58]:
# Add labels to the training set dataframe
train_df["clusters"] = cluster_labels

In [59]:
train_df

,institute,course_id,userid_DI,certified,gender_numerical,age_indicator_0,age_indicator_1,age_indicator_2,age_indicator_4,education_Bachelor's,...,region_Philippines,region_Poland,region_Portugal,region_Russian Federation,region_Spain,region_Ukraine,region_United Kingdom,region_United States,region_Unknown/Other,clusters
0,MITx,6.00x,M130414773,0,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,57
1,MITx,3.091x,M130240725,0,1,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,7
2,HarvardX,PH207x,H130082344,0,1,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,31
3,MITx,6.00x,M130442605,0,1,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,15
4,HarvardX,CB22x,H130226407,0,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314971,MITx,3.091x,M130241907,0,1,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,6
314972,MITx,7.00x,M130241907,1,1,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,6
314973,MITx,6.002x,M130205207,1,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,13
314974,MITx,7.00x,M130205207,1,1,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,13


# Predict User Cluster on Test Data #

In [60]:
# Check test_df columns
test_df.columns

Index(['institute', 'course_id', 'userid_DI', 'certified', 'gender_numerical',
       'age_indicator_0', 'age_indicator_1', 'age_indicator_2',
       'age_indicator_4', 'education_Bachelor's', 'education_Doctorate',
       'education_Less than Secondary', 'education_Master's',
       'education_Secondary', 'region_Australia', 'region_Bangladesh',
       'region_Brazil', 'region_Canada', 'region_China', 'region_Colombia',
       'region_Egypt', 'region_France', 'region_Germany', 'region_Greece',
       'region_India', 'region_Indonesia', 'region_Japan', 'region_Mexico',
       'region_Morocco', 'region_Nigeria', 'region_Other Africa',
       'region_Other East Asia', 'region_Other Europe',
       'region_Other Middle East/Central Asia',
       'region_Other North & Central Amer., Caribbean', 'region_Other Oceania',
       'region_Other South America', 'region_Other South Asia',
       'region_Pakistan', 'region_Philippines', 'region_Poland',
       'region_Portugal', 'region_Russian Fed

In [61]:
# Predict cluster labels
predicted_labels = kmeans.predict(test_df.iloc[:,4:])

# Show labels
predicted_labels

array([57,  6, 34, ...,  9,  9,  9], dtype=int32)

In [62]:
# Add cluster labels to dataframe
test_df["predicted_labels"] = predicted_labels
test_df = test_df.reset_index(drop=True)

In [63]:
# # Check distribution
# sns.histplot(predicted_labels)

In [64]:
# test_pt.loc["M130018888", "sum"]

In [65]:
testids = ["H130128087", "H130228983", "M130018888", "H130588778", "M130136560","M130228447","M130338811", "M130502892"] # 2 instances

In [66]:
test_pt[test_pt["sum"]==1]

course_id,14.73x,2.01x,3.091x,6.002x,6.00x,7.00x,8.02x,8.MReV,CB22x,CS50x,ER22x,PH207x,PH278x,sum
userid_DI,,,,,,,,,,,,,,
H130000071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
H130000715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
H130000944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
H130001577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
H130002019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M130595975,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
M130596958,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
M130597250,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


# Define Recommender Functions #

In [67]:
# test_df[test_df["userid_DI"]=="M130582899"]

In [68]:
# Import library for calculating distance between users
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
def get_user_info(userid, known_user_list=list(test_df["userid_DI"]), known_user_course_matrix=test_pt):
    
    user_info = {}
    
#     known_user_list = list(test_df["userid_DI"])     # For the purposes of testing, limit the search to users in the test_df
#     known_user_course_matrix = test_pt               #   The fully-implmented version of the system would maintian a link
#                                                      #   to the entire database and searc in it
    
    
    # If user is among known users, return user vector
    if userid in known_user_list:
        
        # Get user vector and add it to the user record
        row_idx = test_df[test_df["userid_DI"]==userid].index[0]  # if multiple users are found for a user, return only the first index as all the user vector would be the same in all records
#         print(row_idx)
        user_info["user_vector"] = np.array(test_df.loc[row_idx][4:-1]).reshape(1, -1) # reshape array for further processing
        
        # Get cluster information and add it to the record
        cluster_num = get_cluster(user_info["user_vector"])
        user_info["cluster"] = cluster_num
        
        # Look up user in the "Users and Courses" matrix to check if user
        #  has previously completed any courses
#         n_courses = known_user_course_matrix.loc[userid, "sum"]
        
        # Add course information
        courses = {}
        for course in known_user_course_matrix.columns:
            courses[course] = known_user_course_matrix.loc[userid, course]
        
        user_info["courses"] = courses
    
    # If the user is not a known user, the object will be return empty
    return user_info


def get_unique_sorted_user_list (sorted_user_list):
    '''
        Transform a sorted user list with duplicates to a sorted user list that contains only unique values.
        Sorting function preserves user's first placement.
        
        Input:
            - sorted_user_list  : a list of users that have been sorted
        
        Output:
            - unique_sul        : a list of users in which the duplicate values have been removed but that preserves the original user order
    '''
    
     # Eliminate duplicate entries in sorted ID list
    unique_sul = []
    for i in range(len(sorted_user_list)):
        if sorted_user_list[i] not in unique_sul:  # preserve placement of a similar user's first placement
            unique_sul.append(sorted_user_list[i])
#     print(f"UNIQUE SUL: {unique_sul}")
    return unique_sul

def filter_course_completion_table(ranked_users_list, course_completion_table=train_pt):
    '''
        Filter course completion table in accordance with a ranked user list. The course completeion table 
        is filtered in two ways:
            1) Users who have not completed any courses are removed altogether because their records lack the basis for recommnedation
            2) Remaining users are sorted in accordance with the ranked list (which is itself ranked on the basis of cosine similarity)
        The filtered table is then returned for further processing by the recommender function
        
        Inputs:
          -  ranked_user_list         :  a list of user IDs whose order represents a rank among the users
          -  course_completion_table  :  
    '''
        
    # Filter course completion table
    #   N.B. filtering does not preserve order
    filtered_cct = course_completion_table.iloc[course_completion_table.index.isin(ranked_users_list)]

    # Drop rows that do not contain any completed courses
    filtered_cct = filtered_cct.loc[filtered_cct["sum"]!=0]

    # Create a column to store rank information
    filtered_cct["rank"] = np.zeros(len(filtered_cct))

    # Add rank information to filtered course completion table
    for user in filtered_cct.index:
        filtered_cct.loc[filtered_cct.index==user,"rank"] = ranked_users_list.index(user)    # gaps between users are of no importance because relative rank is preserved


    # Sort users by rank
    filtered_cct.sort_values(["rank"], inplace=True)

#         filtered_cct.set_index("rank", inplace=True)
    filtered_cct = filtered_cct.reset_index()    # new index becomes the ordered and continuous rank of relevant users in the cct

#     print(filtered_cct)

    return filtered_cct


In [70]:
def get_top_n_recs(filtered_table, n):
    '''
        Function that returns the top n recommendations for a user by isolating all the completed courses
        in a given cluster and calculating the completion rate for each of them.
        
        Input:
        -  filtered_table  :  a course completion table filtered by clusters
        -        n         :  the number of recommendations to return
    '''
    
#     print(filtered_table)
#     print(f"cols: {filtered_table.columns[1:-1]}")
    offset = 1 # column offset for further filtering table
    
    # Calculate recommender percentages for all courses
    # Create variables to store information
    course_tally = {}
    count = []
    
    # Find the number of people who completed each course
    for course in filtered_table.columns[offset:-2]:
#         print(f"course: {course}")
        s = np.nansum(filtered_table[course])
        count.append(s)
#         print(f"count: {s}")
    
    # Sort counts to produce rank
    sorted_counts_idx = np.argsort(count)[::-1]
#     print(f"sorted count idx: {sorted_counts_idx}")
    
    # Calculate total
    total = sum(count)

    # Create count object to return
    for i in range(len(sorted_counts_idx)):
        course = filtered_table.columns[sorted_counts_idx[i]+offset]
        val = count[sorted_counts_idx[i]]
        percentage = val/total
        rank = i+1
        course_tally[course] = tuple([val, rank, percentage])
#         print(f"i: {i}, course: {course}, val: {val}")

    
    # Filter top n courses
    top_n_courses = {} 
    
    for course in course_tally.keys():
        if(course_tally[course][1] <= n):
            top_n_courses[course] = course_tally[course]
    
    return top_n_courses




In [71]:
def get_n_recs(filtered_table, n, cluster_size):
    '''
        Alternate approach to getting recommendations by drawing on the completion history of the nearest users in sequence. 
        This is a continuation the of the thinking behind the clustering algorithm.

        Inputs:
          - filtered_table  : a course completion table that has been pre-filtered in accordance with requirements
          -        n        : the number of recommendations to return

        Outputs:
          -      recs       : dictionary of recommendations

    '''
    recs = set()
    count = 0
  
    while (len(recs)<n and count<len(filtered_table)):

            # Isolate row in pivot table
            row = filtered_table.iloc[count, :-2].values

            # Store course indexes for completed courses
            course_idxs = np.where(row == 1.)[0]

            # Find successfully completed courses and add them to the list
            course_idxs = np.where(row == 1.)[0] # isolate first dimension
            
            # Add courses to recommended list
            for idx in course_idxs:
                    recs.add(filtered_table.columns[idx]) 

            count += 1
            
    # Check if the number of recs is n
    #  Mitigate situation when more than one course is added at a time
    #  on account of a user having completed multiple courses
    if len(recs) > n:
        recs = list(recs)[:n]
    
    rec_values = {}
    # Calculate percentage of occurence within cluster
    for course in recs:
#         print(np.nansum(filtered_table[course]))
        rec_values[course] = np.nansum(filtered_table[course])/cluster_size
    
    return rec_values

In [76]:
def get_multi_course_stats(courses_taken, filtered_table):
    
    '''
        A function that deals specifically with "Tier 1" users who have already completed courses. These users 
        receive recommendations based on the courses that co-occur within a cluster alongside the completed courses.
    
        Inputs:
            -  courses_taken   :   a list of courses completed by the user
            -  filtered_table  :   a table containing all relevant records within a cluster
        
        Outputs:
            -     stats        :  object containing the support, confidence and lift measures for each of the 
    '''
        
    # Remove all columns containing only NaN values to make calculations more efficient
    filtered_table = filtered_table.dropna(axis='columns',how='all')
    
    # Number of records in the filtered table
    n_records = len(filtered_table)
    
    # Isolate list of courses to check by removing the completed courses
    courses = set(filtered_table.columns[1:-2]).difference(set(courses_taken))
#     print(filtered_table.columns)
#     print(courses)
    
#     print(set(courses).difference(set(courses_taken)))
    # Define stats object
    stats = {}
#     filter_indices = []
    
#     print(courses)
    
    # Gather information about completed courses and 
    #   remove them from the list of courses
    for course in courses_taken:
#         print(course)
        
        # Count the number of students who have taken the course
        count = np.nansum(filtered_table[course])
        
        # Record indices for the course
        indices = list(filtered_table[filtered_table[course]==1].index)

        
        # Reduce table to the indices containing the current completed course
        reduced_table = filtered_table[filtered_table.index.isin(indices)]    
        
        # Calculate per-course statistics
        for potential_course in courses:
#             print(potential_course)
            # Count the co-ocurrences with each course 
            n_coocurrences = np.nansum(reduced_table[potential_course])
#             print(f"n co-ocurences: {n_coocurrences}")
            
            # Bypass calculating information if there are no co-ocurrences
            if (n_coocurrences==0):
                pass
            
            else:
                # Calculate support: n_cooccurences/ n_records      
                support = n_coocurrences/ n_records

                # Calculate confidence: n_cooccurences/ count
                confidence = n_coocurrences/count

                # Calculate lift: confidence / (n_potential_course/n_records)
                n_potential_course = np.nansum(filtered_table[potential_course]) # number of times potential course appears in cluster
                lift = confidence / (n_potential_course/n_records)

                # Add information to course object
                stats[potential_course]= {
                                            "support": support,
                                            "confidence": confidence,
                                            "lift": lift
                                        }
        
        return stats
    

In [82]:
# Define functions to isolate top n closest datapoints in the appropriate cluster for every record

def get_cluster (user_vector, estimator=kmeans):
    '''
        Funcion that takes a vector containing demographic data from a user and returns the predicted cluster for that user
        
        Input:
          -   user vector  : vector representing user's demographic information expressed as a one-hot encoded vector
          -   estimator    : clustering mechanism
    '''
    
    return estimator.predict(user_vector)[0]

def get_similar_users (user_vector, cluster_num, df=train_df):
    '''
        Function that takes a user vector containing demographic information and returns the distances from every user in the cluster to the test user
        
        Inputs:
            - user_vector : vector denoting user featuers
            -    df       : dataframe containing users and cluster information
            
        Output:
            - sorted_indices : list of indices, sorted in reverse order, denoting the cosine distances from the test users to other users in the cluster
            -   df_subset    : return the subset of the dataframe containing the cluster for future calculations
        
    '''

    # Isolate all items in the cluster
    df_subset = df[df["clusters"]==cluster_num]
#     print(df_subset)
#     print(df_subset.index)
    
    # Calculate cosine distances from the test user to each user in the cluster ignoring cluster information included in dataframe
    cosine_distances = cosine_similarity(user_vector, df_subset.iloc[:,4:-1])[0] # unest array
    
    # Sort distances in reverse order
    sorted_indices = np.argsort(cosine_distances)[::-1]
#     print(df_subset.iloc[sorted_indices[0]])

    # Create an ordered list of users
    ordered_user_list = [df_subset.iloc[sorted_indices[i],2] for i in range(len(sorted_indices))]
    
    
    
    return ordered_user_list

def tabulate_results_CF(user):
    pass

In [72]:
def CF_recommender (userid, df=train_df, course_completion_table=train_pt, n=5, testset=pd.DataFrame()):  
    
    '''
        Main CF recommender function that coordinates the work of all the helper functions. The function 
        fetches user information and the cluster data. Depending on the target user's engagement with 
        the platform, it chooses which recommender mechanism to use.
        
        Input:
        -          userid           :  ID of user on the system
        -           df              :  the main user dataframe. (In this case, it is the train dataframe, which is to be filtered in accordance with the cluster.)
        - course_completion_table   :  the main course completion table, to be flitered by cluster useers
        -           n               :  the number of recommendations to issue
        -         testset           :  testset to be fed in for batch testing
        
        Outputs:
        -         recommendations   :  each subset of the funciton returns its own appropriate type of recommendation

    '''
    
    
    # STEP 1: Get user vector and cluster information
    if (testset.empty):
        user_info = get_user_info(userid)
    else:
        user_info = get_user_info(userid, known_user_course_matrix=testset)

    
    # STEP 2: Check if the user is a known user and, if so,
    #  gather information about similar users
    if (user_info != {}):
        
        # Get a list of users in the cluster sorted by their similarity to the current user
        similar_users_sorted = get_similar_users(user_info["user_vector"], user_info["cluster"])

        # Check for and eliminate duplicates among the sorted users
        if len(set(similar_users_sorted)) < len(similar_users_sorted):

            ranked_users = get_unique_sorted_user_list(similar_users_sorted)

        else:
            ranked_users = similar_users_sorted

        # Filter course compeltion table based on ranked users
        filtered_table = filter_course_completion_table(ranked_users)

        
    
        # STEP 3: Verify user course history and gauge which recommender approach to use
#   
    
        # Create place-holder for recommended courses
        recommended_courses = {}
        
        
        # If user has not completed any courses, generate recommendations 
        # based on the most popular courses in the cluster
        if (user_info["courses"]["sum"] == 0):
            recommended_courses = get_top_n_recs(filtered_table, n)
            return recommended_courses

        else:
            
            # Designate list to hold completed courses
            completed_courses = []
            
            # Populate list
            for course in list(user_info["courses"].keys())[:-1]:
                if (user_info["courses"][course]==1.):
                    completed_courses.append(course)
                
#             print(f"completed courses: {completed_courses}")
            
            
            # -------------------- TIER 1 RECOMMENDATIONS -------------------- #
            
            tier_1_recs = get_multi_course_stats(completed_courses, filtered_table)
#             print(f"tier 1: {tier_1_recs}")
            
            for rec in tier_1_recs:
#                 print(f"rec: {rec}")
                recommended_courses[rec]=tier_1_recs[rec]["confidence"]
                
            # -------------------- TIER 2 RECOMMENDATIONS -------------------- #
            
            # Extract indices relevant rows in the filtered table
            #  and store them in a list
            indices = []
            for course in completed_courses:
                indices.extend(filtered_table.loc[filtered_table[course]==1.].index)

            indices = sorted(list(set(indices)))
#             print(f"indices: {indices}")
            # Filter course completion table to include only user who have taken the same courses
            #  Create a copy of the list to ensure data integrity
            rank_table = filtered_table[filtered_table.index.isin(indices)].copy()

            # Remove the courses the user has completed from the table
            rank_table.drop(completed_courses, axis=1, inplace=True)


            # Get total number of ratings
            total_ratings = np.nansum(rank_table.iloc[:,1:-2])

            # Define variable to store course tally
            course_percentages = {}
    #         print(rank_table.columns)
            # Store course number and the percentage of completions
            for course in rank_table.columns[1:-2]:
                # Prevent duplications and/or overwriting
                if course not in recommended_courses.keys():
                    n_completions = np.nansum(rank_table[course])
                    if (n_completions > 0):
                        course_percentages[course]=n_completions/len(rank_table)
# 

            # Sort dictionary by value
            #  Code based on Stack Overflow entry:
            #  https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
            ranked_courses = sorted(course_percentages, key=course_percentages.get, reverse=True)

            for i in range(len(ranked_courses)):
                recommended_courses[ranked_courses[i]] = course_percentages[ranked_courses[i]]

#             print(f"rec step 1: {recommended_courses}")

            # Check if there are enough courses recommended
            if (len(recommended_courses.items()) >= n):
                recommended_courses = dict(itertools.islice(recommended_courses.items(), n))
                
                return recommended_courses

            else: 
            
            # -------------------- TIER 3 RECOMMENDATIONS -------------------- #
            # RETURN TO CLUSTER
            
                courses_to_remove = completed_courses + list(recommended_courses.keys())
                filtered_table = filtered_table.drop(courses_to_remove, axis=1)

                additional_recommended_courses = get_n_recs(filtered_table, n-len(recommended_courses.items()), len(ranked_users))
#                 print(f"rec step 2: {additional_recommended_courses}")

                return {**recommended_courses, **additional_recommended_courses}
            
    # If user is not know, return empty object
    else:
        return {}
    